### Audio Event Classifier with Deep Learning

Build a CNN sound classifier using melspectograms from ESC-50 data. Refer to *save_melspectorgrams.ipynb* for feature extraction.

In [1]:
%matplotlib inline
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.applications.vgg16 import preprocess_input
from keras.utils import multi_gpu_model
import numpy as np
import json

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Set parameter values

In [2]:
batch_size = 40
epochs = 200

# dimensions of our images.
img_width, img_height = 224, 224

input_tensor = Input(shape=(224,224,3))

nb_training_samples = 1600
nb_validation_samples = 400# Set parameter values

# Configure training and validation data generators

Provide paths to training and testing set directores

In [3]:
# training generator configuration
training_data_dir = '/path/to/ESC-50-master/melspectrograms/training'

training_datagen = image.ImageDataGenerator(
    rescale=1./255)

training_generator = training_datagen.flow_from_directory(
    training_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size)

# validation generator configuration
validation_data_dir ='/path/to/ESC-50-master/melspectrograms/testing/'

validation_datagen = image.ImageDataGenerator(
    rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size)

Found 1600 images belonging to 50 classes.
Found 400 images belonging to 50 classes.


# Load base model

In [4]:
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
print('Model loaded.')
base_model.summary()

Model loaded.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0      

# Build top model

In [5]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(50, activation='softmax'))
top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6422784   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                12850     
Total params: 6,435,634
Trainable params: 6,435,634
Non-trainable params: 0
_________________________________________________________________


# Combine base model with top model

In [6]:
# top_model.load_weights('bootlneck_fc_model.h5')
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

# Configure model training

In [7]:
num_layers_to_freeze = 15

In [8]:
from keras import metrics, optimizers

def top_5_accuracy(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=5)

for layer in model.layers[:num_layers_to_freeze]:
    layer.trainable = False

# use nesterov accelrated gradient descent ??
# optimizer=optimizers.SGD(lr=1e-4, momentum=0.9, decay=1e-6, nesterov=True)
model.compile(optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), 
                      loss='categorical_crossentropy', 
                      metrics=['accuracy', top_5_accuracy])

# parallel_model.compile(optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), 
#                       loss='categorical_crossentropy', 
#                       metrics=['accuracy', top_5_accuracy])

# serialize model to JSON
model_json = model.to_json()
model_filename = "vgg16_model_{}_frozen_layers.json".format(num_layers_to_freeze)
with open(model_filename, "w") as json_file:
    json_file.write(model_json)

# Fine-tune the model

In [9]:
from keras.callbacks import ModelCheckpoint, TensorBoard
from time import time

tensorboard = TensorBoard(log_dir="logs/layers_frozen_{}".format(num_layers_to_freeze))

# checkpoint
filepath="esc50_vgg16_stft_weights_train_last_2_base_layers.best.hdf5"
best_model_checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [best_model_checkpoint, tensorboard]

# parallel_model.fit_generator(
#     training_generator,
#     steps_per_epoch=nb_training_samples/batch_size,
#     epochs=epochs,
#     validation_data=validation_generator,
#     validation_steps=nb_validation_samples/batch_size,
#     callbacks=callbacks_list)

model.fit_generator(
    training_generator,
    steps_per_epoch=nb_training_samples/batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples/batch_size,
    callbacks=callbacks_list)
# parallel_model.fit_generator(
#     training_generator,
#     samples_per_epoch=nb_training_samples,
#     epochs=epochs,
#     validation_data=validation_generator,
#     validation_steps=nb_validation_samples/batch_size,)
#     nb_val_samples=nb_validation_samples)

Epoch 1/200
40/40 [==============================] - 9s 216ms/step - loss: 4.0415 - acc: 0.0263 - top_5_accuracy: 0.1188 - val_loss: 3.8747 - val_acc: 0.0375 - val_top_5_accuracy: 0.1625

Epoch 00001: val_acc improved from -inf to 0.03750, saving model to esc50_vgg16_stft_weights_train_last_2_base_layers.best.hdf5
Epoch 2/200
40/40 [==============================] - 7s 177ms/step - loss: 3.8761 - acc: 0.0400 - top_5_accuracy: 0.1456 - val_loss: 3.8452 - val_acc: 0.0500 - val_top_5_accuracy: 0.1975

Epoch 00002: val_acc improved from 0.03750 to 0.05000, saving model to esc50_vgg16_stft_weights_train_last_2_base_layers.best.hdf5
Epoch 3/200
40/40 [==============================] - 7s 186ms/step - loss: 3.8432 - acc: 0.0400 - top_5_accuracy: 0.1894 - val_loss: 3.8026 - val_acc: 0.0675 - val_top_5_accuracy: 0.2925

Epoch 00003: val_acc improved from 0.05000 to 0.06750, saving model to esc50_vgg16_stft_weights_train_last_2_base_layers.best.hdf5
Epoch 4/200
40/40 [===========================

40/40 [==============================] - 7s 186ms/step - loss: 1.7880 - acc: 0.5013 - top_5_accuracy: 0.8256 - val_loss: 1.6833 - val_acc: 0.5575 - val_top_5_accuracy: 0.8725

Epoch 00029: val_acc did not improve
Epoch 30/200
40/40 [==============================] - 7s 182ms/step - loss: 1.6263 - acc: 0.5656 - top_5_accuracy: 0.8688 - val_loss: 1.6043 - val_acc: 0.5950 - val_top_5_accuracy: 0.8950

Epoch 00030: val_acc improved from 0.57500 to 0.59500, saving model to esc50_vgg16_stft_weights_train_last_2_base_layers.best.hdf5
Epoch 31/200
40/40 [==============================] - 7s 183ms/step - loss: 1.5498 - acc: 0.5856 - top_5_accuracy: 0.8763 - val_loss: 1.5558 - val_acc: 0.6000 - val_top_5_accuracy: 0.9075

Epoch 00031: val_acc improved from 0.59500 to 0.60000, saving model to esc50_vgg16_stft_weights_train_last_2_base_layers.best.hdf5
Epoch 32/200
40/40 [==============================] - 8s 188ms/step - loss: 1.5791 - acc: 0.5594 - top_5_accuracy: 0.8819 - val_loss: 1.5205 - val_

40/40 [==============================] - 7s 187ms/step - loss: 0.6031 - acc: 0.8219 - top_5_accuracy: 0.9856 - val_loss: 1.0224 - val_acc: 0.7075 - val_top_5_accuracy: 0.9300

Epoch 00060: val_acc did not improve
Epoch 61/200
40/40 [==============================] - 7s 183ms/step - loss: 0.5796 - acc: 0.8444 - top_5_accuracy: 0.9819 - val_loss: 0.9625 - val_acc: 0.7075 - val_top_5_accuracy: 0.9425

Epoch 00061: val_acc did not improve
Epoch 62/200
40/40 [==============================] - 7s 184ms/step - loss: 0.5454 - acc: 0.8481 - top_5_accuracy: 0.9863 - val_loss: 0.9752 - val_acc: 0.7150 - val_top_5_accuracy: 0.9325

Epoch 00062: val_acc did not improve
Epoch 63/200
40/40 [==============================] - 7s 187ms/step - loss: 0.5681 - acc: 0.8537 - top_5_accuracy: 0.9825 - val_loss: 0.9893 - val_acc: 0.7100 - val_top_5_accuracy: 0.9400

Epoch 00063: val_acc did not improve
Epoch 64/200
40/40 [==============================] - 7s 181ms/step - loss: 0.5146 - acc: 0.8594 - top_5_accu


Epoch 00093: val_acc did not improve
Epoch 94/200
40/40 [==============================] - 7s 181ms/step - loss: 0.2405 - acc: 0.9450 - top_5_accuracy: 0.9988 - val_loss: 0.8553 - val_acc: 0.7300 - val_top_5_accuracy: 0.9425

Epoch 00094: val_acc did not improve
Epoch 95/200
40/40 [==============================] - 7s 184ms/step - loss: 0.2290 - acc: 0.9506 - top_5_accuracy: 0.9994 - val_loss: 0.8733 - val_acc: 0.7200 - val_top_5_accuracy: 0.9475

Epoch 00095: val_acc did not improve
Epoch 96/200
40/40 [==============================] - 7s 186ms/step - loss: 0.2451 - acc: 0.9381 - top_5_accuracy: 0.9988 - val_loss: 0.8524 - val_acc: 0.7225 - val_top_5_accuracy: 0.9450

Epoch 00096: val_acc did not improve
Epoch 97/200
40/40 [==============================] - 7s 181ms/step - loss: 0.2416 - acc: 0.9394 - top_5_accuracy: 0.9975 - val_loss: 0.8891 - val_acc: 0.7225 - val_top_5_accuracy: 0.9475

Epoch 00097: val_acc did not improve
Epoch 98/200
40/40 [==============================] - 7s 1


Epoch 00128: val_acc did not improve
Epoch 129/200
40/40 [==============================] - 7s 182ms/step - loss: 0.1331 - acc: 0.9706 - top_5_accuracy: 1.0000 - val_loss: 0.8315 - val_acc: 0.7525 - val_top_5_accuracy: 0.9450

Epoch 00129: val_acc did not improve
Epoch 130/200
40/40 [==============================] - 7s 182ms/step - loss: 0.1235 - acc: 0.9731 - top_5_accuracy: 0.9994 - val_loss: 0.8335 - val_acc: 0.7575 - val_top_5_accuracy: 0.9400

Epoch 00130: val_acc did not improve
Epoch 131/200
40/40 [==============================] - 7s 184ms/step - loss: 0.1208 - acc: 0.9744 - top_5_accuracy: 0.9994 - val_loss: 0.8416 - val_acc: 0.7650 - val_top_5_accuracy: 0.9375

Epoch 00131: val_acc improved from 0.75750 to 0.76500, saving model to esc50_vgg16_stft_weights_train_last_2_base_layers.best.hdf5
Epoch 132/200
40/40 [==============================] - 7s 184ms/step - loss: 0.1055 - acc: 0.9794 - top_5_accuracy: 1.0000 - val_loss: 0.8417 - val_acc: 0.7600 - val_top_5_accuracy: 0.937

40/40 [==============================] - 7s 185ms/step - loss: 0.0537 - acc: 0.9913 - top_5_accuracy: 1.0000 - val_loss: 0.8662 - val_acc: 0.7550 - val_top_5_accuracy: 0.9350

Epoch 00200: val_acc did not improve


# Get top k predictions for selected test files

In [10]:
def get_top_k_predictions(preds, label_map, k=5, print_flag=False):
    sorted_array = np.argsort(preds)[::-1]
    top_k = sorted_array[:k]
    label_map_flip = dict((v,k) for k,v in label_map.iteritems())
    
    y_pred = []
    for label_index in top_k:
        if print_flag:
            print "{} ({})".format(label_map_flip[label_index], preds[label_index])
        y_pred.append(label_map_flip[label_index])
        
    return y_pred

In [11]:
label_map = (training_generator.class_indices)
 
json = json.dumps(label_map)
f = open("cough_label_map.json","w")
f.write(json)
f.close()

img_path = '/data/datasets/ESC-50/ESC-50-master/spectrograms/testing/vacuum_cleaner/5-182007-A-36.png'

img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)* 1./255

preds = model.predict(x)[0]

get_top_k_predictions(preds, label_map, k=3)

['vacuum_cleaner', 'crickets', 'engine']

# Calculate and plot confusion matrix

In [ ]:
import itertools
import matplotlib.pyplot as plt
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

#     print(cm)
    plt.figure(figsize=(24,24))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

In [ ]:
import os
from sklearn.metrics import confusion_matrix

testing_dir = '/data/datasets/ESC-50/ESC-50-master/melspectrograms/testing/'

y_true = []
y_pred = []
for label in label_map.keys():
    file_list = os.listdir(testing_dir + label)
    for file_name in file_list:
        img_path = testing_dir + label + '/' + file_name
        
        img = image.load_img(img_path, target_size=(224, 224))
        
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)* 1./255
        
        preds = model.predict(x)[0]
        
        y_true.append(label)
        y_pred.append(get_top_k_predictions(preds, label_map, k=1)[0])
        
cm = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cm, sorted(label_map.keys()), normalize=True)